In [1]:
##universal setting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import shapely
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns

In [2]:
#purpose 
testing="price_less_350_more_LassoFeatureSelection"
price_max=350
# data location
read_location="C:/Selina/Class/(DS_MS)/Capstone/DataSet/"
save_location="C:/Selina/Class/(DS_MS)/Capstone/Result/"
#plot color setting (https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html)
my_cmap=plt.cm.get_cmap("Pastel2")
#for geopandas setting
crs={'init': 'epsg:4326'}

In [3]:
#import data
data=pd.read_csv(read_location+"AB_NYC_2019_GIS_km_all.csv",index_col=False)
#data = data.reset_index()
print(data.columns)
#define the name of algebra
y="price"

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'distance to the nearest metro',
       'number of museum within 500', 'distance to the nearest popular scene',
       'number of top10 within 500', 'NTAName', 'review this year'],
      dtype='object')


In [4]:
#variables of categorical data 
x_rt="room_type"
enc_rt= OneHotEncoder(handle_unknown='ignore')
a=enc_rt.fit_transform(data[x_rt].values.reshape((-1,1))).toarray()
df_a=pd.DataFrame(a, columns=enc_rt.categories_[0])
data = pd.concat([data, df_a], axis=1)

x_ng="neighbourhood_group"
enc_ng= OneHotEncoder(handle_unknown='ignore')
a=enc_ng.fit_transform(data[x_ng].values.reshape((-1,1))).toarray()
df_a=pd.DataFrame(a, columns=enc_ng.categories_[0])
data = pd.concat([data, df_a], axis=1)

x_nt='NTAName'
enc_nt= OneHotEncoder(handle_unknown='ignore')
a=enc_nt.fit_transform(data[x_nt].values.reshape((-1,1))).toarray()
df_a=pd.DataFrame(a, columns=enc_nt.categories_[0])
data = pd.concat([data, df_a], axis=1)
print(data.head(3))
print(data.columns)

     id                                 name  host_id  host_name  \
0  2539   Clean & quiet apt home by the park     2787       John   
1  2595                Skylit Midtown Castle     2845   Jennifer   
2  3647  THE VILLAGE OF HARLEM....NEW YORK !     4632  Elisabeth   

  neighbourhood_group                      neighbourhood  latitude  longitude  \
0            Brooklyn                    Windsor Terrace  40.64749  -73.97237   
1           Manhattan              Midtown-Midtown South  40.75362  -73.98377   
2           Manhattan  Central Harlem North-Polo Grounds  40.80902  -73.94190   

         room_type  price  ...  Williamsburg  Windsor Terrace Woodhaven  \
0     Private room    149  ...           0.0              1.0       0.0   
1  Entire home/apt    225  ...           0.0              0.0       0.0   
2     Private room    150  ...           0.0              0.0       0.0   

   Woodlawn-Wakefield  Woodside  Yorkville  park-cemetery-etc-Bronx  \
0                 0.0       0.

In [5]:
drop_list=[x_ng,x_rt,x_nt,'id', 'name', 'host_id', 'host_name','neighbourhood',"last_review",]
data=data.drop(columns=drop_list)
print(data.head(3))

   latitude  longitude  price  minimum_nights  number_of_reviews  \
0  40.64749  -73.97237    149               1                  9   
1  40.75362  -73.98377    225               1                 45   
2  40.80902  -73.94190    150               3                  0   

   reviews_per_month  calculated_host_listings_count  availability_365  \
0               0.21                               6               365   
1               0.38                               2               355   
2                NaN                               1               365   

   distance to the nearest metro  number of museum within 500  ...  \
0                     465.387704                            0  ...   
1                      93.434732                            3  ...   
2                     334.408650                            1  ...   

   Williamsburg  Windsor Terrace  Woodhaven  Woodlawn-Wakefield  Woodside  \
0           0.0              1.0        0.0                 0.0       0.

In [6]:
#drop price=0
data=data[data[y]!=0]
data=data[data[y]<=price_max]
data = data.reset_index(drop=True)
data.head(3)

,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,distance to the nearest metro,number of museum within 500,...,Williamsburg,Windsor Terrace,Woodhaven,Woodlawn-Wakefield,Woodside,Yorkville,park-cemetery-etc-Bronx,park-cemetery-etc-Brooklyn,park-cemetery-etc-Manhattan,park-cemetery-etc-Queens
0,40.64749,-73.97237,149,1,9,0.21,6,365,465.387704,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,40.75362,-73.98377,225,1,45,0.38,2,355,93.434732,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,40.80902,-73.94190,150,3,0,NaN,1,365,334.408650,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
#fill na of variable
data['reviews_per_month']=data['reviews_per_month'].replace(np.nan, 0)
#fill # of minimun night
median_temp = data.loc[data["minimum_nights"]<=365, "minimum_nights"].median()
data["minimum_nights"] = np.where(data["minimum_nights"] >365, median_temp,data["minimum_nights"])

In [8]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=[y]), data[y], test_size=0.3, random_state=1234)
print(X_train.head(3))
print("--------------")
print(y_train.head(3))

       latitude  longitude  minimum_nights  number_of_reviews  \
42996  40.73209  -73.88142             1.0                  0   
43676  40.71114  -73.96153            14.0                  0   
37264  40.63912  -73.92827             2.0                 17   

       reviews_per_month  calculated_host_listings_count  availability_365  \
42996               0.00                               1                 0   
43676               0.00                               1               139   
37264               2.48                               2                 0   

       distance to the nearest metro  number of museum within 500  \
42996                     633.147346                            0   
43676                     437.478551                            0   
37264                    1697.375475                            0   

       distance to the nearest popular scene  ...  Williamsburg  \
42996                            8055.300961  ...           0.0   
43676          

In [9]:
#standardization for variables 
x_nfr="number_of_reviews"
scaler_nfr= StandardScaler()
scaler_nfr.fit(X_train[x_nfr].values.reshape((-1,1)))
X_train[x_nfr]=scaler_nfr.transform(X_train[[x_nfr]])
X_test[x_nfr]=scaler_nfr.transform(X_test[[x_nfr]])
print(X_train[x_nfr].head(5))

#a=scaler_nfr.transform(X_train[[x_nfr]])
#X_train[x_nfr]=a
#X_train.shape

x_rpm="reviews_per_month"
scaler_rpm= StandardScaler()
scaler_rpm.fit(X_train[x_rpm].values.reshape((-1,1)))
X_train[x_rpm]=scaler_rpm.transform(X_train[[x_rpm]])
X_test[x_rpm]=scaler_rpm.transform(X_test[[x_rpm]])

x_chlc="calculated_host_listings_count"
scaler_chlc= StandardScaler()
scaler_chlc.fit(X_train[x_chlc].values.reshape((-1,1)))
X_train[x_chlc]=scaler_chlc.transform(X_train[[x_chlc]])
X_test[x_chlc]=scaler_chlc.transform(X_test[[x_chlc]])

x_a3="availability_365"
scaler_a3= StandardScaler()
scaler_a3.fit(X_train[x_a3].values.reshape((-1,1)))
X_train[x_a3]=scaler_a3.transform(X_train[[x_a3]])
X_test[x_a3]=scaler_a3.transform(X_test[[x_a3]])


42996   -0.529445
43676   -0.529445
37264   -0.148022
20071   -0.305079
20185    1.826399
Name: number_of_reviews, dtype: float64


In [10]:
#a=scaler_nfr.transform(X_train[[x_nfr]])
#X_train[x_nfr]=a
#X_train.shape
print(X_test.columns)
NT_name=list(X_test.columns[-191:])

Index(['latitude', 'longitude', 'minimum_nights', 'number_of_reviews',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'distance to the nearest metro',
       'number of museum within 500', 'distance to the nearest popular scene',
       ...
       'Williamsburg', 'Windsor Terrace', 'Woodhaven', 'Woodlawn-Wakefield',
       'Woodside', 'Yorkville', 'park-cemetery-etc-Bronx',
       'park-cemetery-etc-Brooklyn', 'park-cemetery-etc-Manhattan',
       'park-cemetery-etc-Queens'],
      dtype='object', length=211)


In [11]:
all_columns=[
    'latitude', 'longitude', 'minimum_nights', 'number_of_reviews','reviews_per_month', 'calculated_host_listings_count','availability_365',
    'Entire home/apt','Private room', 'Shared room', 'Bronx', 'Brooklyn', 'Manhattan','Queens', 'Staten Island',
    'distance to the nearest metro','number of museum within 500', 'distance to the nearest popular scene','review this year','number of top10 within 500'
]+NT_name

Lasso_columns=['minimum_nights', 'number_of_reviews', 'reviews_per_month','calculated_host_listings_count', 'availability_365',
               'Entire home/apt','Manhattan',
               'distance to the nearest metro','number of museum within 500', 'distance to the nearest popular scene',
               'North Side-South Side']

testing_columns=Lasso_columns

X_train.loc[:,testing_columns].to_csv(read_location+"X_train.csv",index=False)
X_test.loc[:,testing_columns].to_csv(read_location+"X_test.csv",index=False)
pd.DataFrame(y_train).loc[:,[y]].to_csv(read_location+"y_train.csv",index=False)
pd.DataFrame(y_test).loc[:,[y]].to_csv(read_location+"y_test.csv",index=False)

In [12]:
print(pd.DataFrame(y_test))
print("Requirement # of columns:",len(testing_columns),"------","output # : ", len(X_test.columns))

       price
14657     42
7198      50
39264     75
27056     38
20237     79
...      ...
41628    300
42742    140
37344    200
9539     190
7666      70

[13925 rows x 1 columns]
Requirement # of columns: 11 ------ output # :  211


In [13]:
#X_train.describe().to_csv(save_location+testing+"_Xtrain_Summary.csv")